In [ ]:
# importing necessary libraries 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision 
from torchvision import transforms, datasets, models 
from torch.nn import functional as F 
from PIL import Image
import pandas as pd 
import numpy as np
import tensorflow as tf
import os
import sys
import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt 
import cv2
import json 
from tqdm import tqdm
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from skimage.transform import rotate, AffineTransform
import random
from scipy import ndimage
import openslide
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import xml.etree.ElementTree as ET 
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report
import seaborn as sns 
import gc 
import math 
from collections import defaultdict 
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math 
import matplotlib 
import collections 
from scipy.ndimage import label 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
patch_size = 0 

weights_dir = 'model_weights_dir' 
model_weights_pths = [f'{weights_dir}/{file_name}' for file_name in os.listdir(weights_dir)] 
model_weights_pths.sort() 
model_weights_pths 

**WSI DIMENSIONS** 

In [ ]:
# dictionary for width and height 
dimensions_dict = {'A01': (62625, 42113), 'A02': (54721, 46305), 'A03': (57345, 38945), 'A04': (64703, 45808), 
                   'A05': (51553, 42625), 'A06': (62625, 36833), 'A07': (50529, 36833), 'A08': (58401, 41057), 
                   'A09': (64191, 34700), 'A10': (58401, 39959)} 

**OBTAINING PATCHES AND PATCH LEVEL LABELS** 

In [ ]:
# Function to obtain patches and their labels against annotation matrices 

def get_all_patches_info(df): 
    
    wsi_nos = df['WSI No.'].to_list() 
    x_pixels = df['X Pixel'].to_list() 
    y_pixels = df['Y Pixel'].to_list() 
    updated_labels = df['Exact Label'].to_list() 
    patch_paths = [f'/all_wsi_patches_{patch_size}/' + str(wsi_nos[i]) + '_' + str(x_pixels[i]) + '_' + str(y_pixels[i]) + '.jpg' for i in range(len(wsi_nos))] 

    return wsi_nos, patch_paths, updated_labels, x_pixels, y_pixels 

In [ ]:
# Obtaining separate datasets for 5-fold cv testing 

def get_dataset(test_df): 
    
    # obtaining patch infos for all wsis 
    wsi_ids, patch_paths, updated_labels_temp, x_pixels, y_pixels = get_all_patches_info(test_df) 
    
    # Encode with binary class labels
    label_to_onehot = {
        'Benign': 0,
        'White': 0
    }
    updated_labels = [label_to_onehot.get(label, 1) for label in updated_labels_temp] 

    # Create instances of custom datasets 
    test_dataset = CustomDataset(patch_paths, updated_labels, x_pixels, y_pixels) 
    return wsi_ids, x_pixels, y_pixels, test_dataset 

**CUSTOM DATASET & DATA LOADER FOR TESTING** 

In [ ]:
from torchvision.transforms import ToTensor

class CustomDataset(Dataset):
    def __init__(self, patches, labels, x_pixels, y_pixels):
        assert len(patches) == len(labels) == len(x_pixels) == len(y_pixels), "Mismatch in number of patches and labels"
        self.patches = patches
        self.labels = labels
        self.x_pixels = x_pixels
        self.y_pixels = y_pixels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Check for valid index range
        if idx >= len(self.labels) or idx >= len(self.patches):
            raise IndexError(f"Index {idx} out of range for dataset with length {len(self)}")

        patch = Image.open(self.patches[idx]).convert("RGB")  # Adjust this if `patches` are file paths
        patch = patch.resize((224, 224))  
        patch = ToTensor()(patch)
        patch = nn.functional.normalize(patch, dim=0, p=2)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return patch, label, self.x_pixels[idx], self.y_pixels[idx]

In [ ]:
batch_size = 128 

In [ ]:
# loading patch data 

df = pd.read_csv(f'/all_patches_info_{patch_size}.csv') 

splits = [[x.split('-')[-2], x.split('-')[-1].split('.')[0]] for x in model_weights_pths] 

test_loaders = [] 
wsi_nos = [] 

for i in range(len(splits)): 
    
    test_df = df[df['WSI No.'].isin(splits[i])] 
    wsis, x_pixels, y_pixels, test_dataset = get_dataset(test_df) 
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 8) 
    test_loaders.append(test_loader) 
    wsi_nos.extend(wsis) 

**MODEL (USING IMPORTED WEIGHTS FROM LAN** 

In [ ]:
from torch.nn import functional as F
import torchvision.models as models

class Pipeline(nn.Module):
    def __init__(self,num_classes):
        #define necessary layers
        super().__init__()
        self.num_classes = num_classes
        self.base = models.efficientnet_b0(weights=None)

        # freeze model weights
        for param in self.base.parameters():
            param.requires_grad = False
        
        self.flatten = nn.Flatten()
        
        self.head = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ) 
        
        # freeze head weights
        for param in self.head.parameters():
            param.requires_grad = False
        
    def forward(self,X):
            X = self.base(X)
            X = self.flatten(X)
            X = self.head(X)
            return X, F.sigmoid(X) 

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

**EVALUATING** 

In [ ]:
test_loss = 0
correct = 0
total = 0
y_true_test = []
y_pred_test = []
all_x_pixels = []
all_y_pixels = []
y_probs = []

for i in range(len(model_weights_pths)): 

    pos_weight = torch.tensor([0.0])
    pos_weight = pos_weight.to(device)
    loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
    model = Pipeline(1) 
    
    # importing weights from LAN ran on BACH B 
    model.load_state_dict(torch.load(model_weights_pths[i])) 
    
    model = model.to(device) 
    optimizer = optim.Adam(params=model.parameters(), lr=0.0) 
    
    model.eval() 
    
    with torch.no_grad():
        for images, labels, x_pixels, y_pixels in tqdm(test_loaders[i]):
            # Move data to GPU if available
            images = images.to(device)
            labels = labels.to(device)
            outputs_without_sigmoid, outputs_with_sigmoid = model(images)
            loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
            test_loss += loss.item()  # L+=l.item()
            predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            y_true_test.extend(labels.cpu().detach().numpy())
            y_pred_test.extend(predicted.cpu().detach().numpy())
            y_probs.extend(outputs_with_sigmoid.cpu().detach().numpy())

            all_x_pixels.extend(x_pixels)
            all_y_pixels.extend(y_pixels)

In [ ]:
test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

# Calculate ROC-AUC for test set
fpr_test, tpr_test, _ = roc_curve(y_true_test, y_probs)
roc_auc_test = auc(fpr_test, tpr_test)

print(f'Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.2f}%')
print(f'Test ROC-AUC: {roc_auc_test:.4f}')

# Classification report for test set
print("Test Classification Report:")
print(classification_report(y_true_test, y_pred_test)) 

In [ ]:
# ROC curve for the test set
plt.figure(figsize=(4, 4))
plt.plot(fpr_test, tpr_test, label=f'Test ROC Curve (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Test Set')
plt.legend()
plt.show()

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_true_test, y_pred_test)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Patch Level Confusion Matrix')
plt.show()

**AGGREGATION OF LABELS**

In [ ]:
wsi_metrics_dict = {} 
accuracies = [] 
pos_f1s = [] 
neg_f1s = [] 
dice_coefs = [] 

In [ ]:
from collections import defaultdict

# Dictionary to store results
wsi_metrics_dict = {}

def calculate_wsi_level_metrics(wsi_ids, ground_labels, predicted_labels):
    # Dictionary to store TP, FP, FN counts per WSI and per class
    wsi_metrics = defaultdict(lambda: {
        "0": {"TP": 0, "FP": 0, "FN": 0},
        "1": {"TP": 0, "FP": 0, "FN": 0}
    })

    # Separate dictionaries for accuracy tracking
    correct_predictions = defaultdict(int)
    total_predictions = defaultdict(int)

    # Also store true/pred labels per WSI to compute weighted F1
    true_by_wsi = defaultdict(list)
    pred_by_wsi = defaultdict(list)

    # Iterate through each patch prediction
    for wsi, ground, pred in zip(wsi_ids, ground_labels, predicted_labels):
        total_predictions[wsi] += 1
        true_by_wsi[wsi].append(ground)
        pred_by_wsi[wsi].append(pred)

        if ground == pred:
            correct_predictions[wsi] += 1

        # Update TP, FP, FN
        if ground == 1 and pred == 1:
            wsi_metrics[wsi]["1"]["TP"] += 1
        elif ground == 1 and pred == 0:
            wsi_metrics[wsi]["1"]["FN"] += 1
            wsi_metrics[wsi]["0"]["FP"] += 1
        elif ground == 0 and pred == 0:
            wsi_metrics[wsi]["0"]["TP"] += 1
        elif ground == 0 and pred == 1:
            wsi_metrics[wsi]["0"]["FN"] += 1
            wsi_metrics[wsi]["1"]["FP"] += 1

    # Calculate metrics per WSI
    for wsi in wsi_metrics:
        accuracy = (correct_predictions[wsi] / total_predictions[wsi]) * 100 if total_predictions[wsi] > 0 else 0

        f1_scores = {}
        for class_label in ["0", "1"]:
            tp = wsi_metrics[wsi][class_label]["TP"]
            fp = wsi_metrics[wsi][class_label]["FP"]
            fn = wsi_metrics[wsi][class_label]["FN"]

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            f1_scores[class_label] = f1

        # Weighted F1 Score (manual version)
        support_0 = true_by_wsi[wsi].count(0)
        support_1 = true_by_wsi[wsi].count(1)
        total_support = support_0 + support_1

        weighted_f1 = 0
        if total_support > 0:
            weighted_f1 = (
                (support_0 * f1_scores["0"] + support_1 * f1_scores["1"]) / total_support
            )

        wsi_metrics_dict[wsi] = (
            f"WSI {wsi}: Accuracy = {accuracy:.2f}%, "
            f"F1 Score Class 0 = {f1_scores['0']:.2f}, "
            f"F1 Score Class 1 = {f1_scores['1']:.2f}, "
            f"Weighted F1 = {weighted_f1:.2f}"
        )

# Call the function
calculate_wsi_level_metrics(wsi_nos, y_true_test, y_pred_test)

In [ ]:
wsi_metrics_dict 

**PREDICTION AND PROBABILITY MAPS**

In [ ]:
matplotlib.use('Agg')  # Use non-interactive backend for speed
from mpl_toolkits.axes_grid1 import make_axes_locatable

num_classes = 2
save_dir_binary = "/kaggle/working/prediction_maps"
os.makedirs(save_dir_binary, exist_ok=True)
save_dir_probs = "/kaggle/working/probability_maps"
os.makedirs(save_dir_probs, exist_ok=True)

curr = prev = wsi_nos[0]
width, height = dimensions_dict[curr]
width = width // patch_size 
height = height // patch_size 
curr_ann_map_binary = np.zeros((height, width), dtype=np.uint8) 
curr_ann_map_probs = np.zeros((height, width), dtype=np.float16) 
curr_ann_map_labels = np.zeros((height, width), dtype=np.uint8) 
kernel = np.ones((2, 2), np.uint8)  # adjust for gap size

all_ann_binary_maps = [] 
all_ann_probs_maps = [] 
all_ann_label_maps = [] 

for i in tqdm(range(len(y_true_test))):
    curr = wsi_nos[i]

    if curr != prev:

        # GROUND LABEL MASKS 
        all_ann_label_maps.append(curr_ann_map_labels) 

        # BINARY 
        # Apply morphological gap closing
        #curr_ann_map_binary = cv2.morphologyEx(curr_ann_map_binary, cv2.MORPH_CLOSE, kernel)
        # Save the accumulated annotation matrix for the previous WSI 
        all_ann_binary_maps.append(curr_ann_map_binary) 
        fig, ax = plt.subplots(figsize=(10, 10))
        cmap = plt.get_cmap('PuRd', num_classes)
        im = ax.imshow(curr_ann_map_binary, cmap=cmap, interpolation='nearest', vmin=0, vmax=num_classes - 1)
        ax.set_title(f'Prediction Map for WSI {prev}')
        ax.axis('off')
        # Shrink the colorbar
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="1.5%", pad=0.03)
        plt.colorbar(im, cax=cax, ticks=range(num_classes), label='Class Index')
        plt.savefig(os.path.join(save_dir_binary, f"{prev}_prediction_map.png"), bbox_inches="tight", dpi=150)
        plt.close()

        # PROBABILITY 
        # Apply morphological gap closing
        #curr_ann_map_probs = cv2.morphologyEx(curr_ann_map_probs, cv2.MORPH_CLOSE, kernel)
        # Save the accumulated annotation matrix for the previous WSI 
        all_ann_probs_maps.append(curr_ann_map_probs) 
        fig, ax = plt.subplots(figsize=(10, 10))
        cmap = plt.get_cmap('viridis')  # continuous colormap
        im = plt.imshow(curr_ann_map_probs, cmap=cmap, interpolation='nearest')
        ax.set_title(f'Probability Map for WSI {prev}')
        ax.axis('off')
        # Shrink the colorbar
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="1.5%", pad=0.03)
        plt.colorbar(im, cax=cax, label='Probability')
        plt.savefig(os.path.join(save_dir_probs, f"{prev}_probability_map.png"), bbox_inches="tight", dpi=150)
        plt.close() 

        # Start next WSI
        width, height = dimensions_dict[curr]
        width = width // patch_size 
        height = height // patch_size 
        curr_ann_map_binary = np.zeros((height, width), dtype=np.uint8) 
        curr_ann_map_probs = np.zeros((height, width), dtype=np.float16) 
        curr_ann_map_labels = np.zeros((height, width), dtype=np.uint8) 

    # Patch assignment
    y_start, y_end = all_y_pixels[i], min(all_y_pixels[i] + patch_size, curr_ann_map_binary.shape[0])
    x_start, x_end = all_x_pixels[i], min(all_x_pixels[i] + patch_size, curr_ann_map_binary.shape[1])
    patch_height = y_end - all_y_pixels[i]
    patch_width = x_end - all_x_pixels[i]

    # Directly fill value without creating a full patch array 
    y_start = y_start // patch_size  
    x_start = x_start // patch_size  
    if (y_start < height) and (x_start < width):
        curr_ann_map_binary[y_start, x_start] = y_pred_test[i] 
        curr_ann_map_probs[y_start, x_start] = y_probs[i] 
        curr_ann_map_labels[y_start, x_start] = y_true_test[i] 
        
    prev = curr

# GROUND LABEL MASKS 
all_ann_label_maps.append(curr_ann_map_labels) 

# BINARY 
# Save the last WSI
#curr_ann_map_binary = cv2.morphologyEx(curr_ann_map_binary, cv2.MORPH_CLOSE, kernel)
all_ann_binary_maps.append(curr_ann_map_binary) 
fig, ax = plt.subplots(figsize=(10, 10))
cmap = plt.get_cmap('PuRd', num_classes)
im = ax.imshow(curr_ann_map_binary, cmap=cmap, interpolation='nearest', vmin=0, vmax=num_classes - 1)
ax.set_title(f'Prediction Map for WSI {prev}')
ax.axis('off')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="1.5%", pad=0.03)
plt.colorbar(im, cax=cax, ticks=range(num_classes), label='Class Index')
plt.savefig(os.path.join(save_dir_binary, f"{prev}_prediction_map.png"), bbox_inches="tight", dpi=150)
plt.close() 

# PROBABILITY 
# Save the last WSI
#curr_ann_map_probs = cv2.morphologyEx(curr_ann_map_probs, cv2.MORPH_CLOSE, kernel)
all_ann_probs_maps.append(curr_ann_map_probs) 
fig, ax = plt.subplots(figsize=(10, 10))
cmap = plt.get_cmap('viridis')  # continuous colormap
im = plt.imshow(curr_ann_map_probs, cmap=cmap, interpolation='nearest')
ax.set_title(f'Probability Map for WSI {prev}')
ax.axis('off')
# Shrink the colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="1.5%", pad=0.03)
plt.colorbar(im, cax=cax, label='Probability')
plt.savefig(os.path.join(save_dir_probs, f"{prev}_probability_map.png"), bbox_inches="tight", dpi=150)
plt.close() 

**DICE COEFFICIENT**

In [ ]:
def dice_coefficient(pred_mask, gt_mask, smooth=1e-5):
    pred = pred_mask.astype(np.bool_)
    gt = gt_mask.astype(np.bool_)

    intersection = np.logical_and(pred, gt).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + gt.sum() + smooth)
    return dice 

for i in range(len(all_ann_binary_maps)): 
    dice_score = dice_coefficient(all_ann_binary_maps[i], all_ann_label_maps[i]) 
    dice_coefs.append(dice_score) 
    wsi_metrics_dict[list(wsi_metrics_dict.keys())[i]] += (f", Dice coefficient = {dice_score:.4f}") 

In [ ]:
wsi_metrics_dict

**CLUSTERING**

In [ ]:
from sklearn.cluster import DBSCAN 

In [ ]:
dice_scores = {}  # Store dice per WSI

dir_path = "dir_to_save_clusters"

for i, binary_mask in enumerate(all_ann_binary_maps):
    wsi_id = wsi_nos[i]
    h, w = binary_mask.shape

    coords = []  # list of (row, col) of tumor patches

    for row in range(h):
        for col in range(w):
            if binary_mask[row, col]:  # tumor region
                coords.append([row, col])

    if len(coords) == 0:
        print(f"{wsi_id}: No tumor patches detected.")
        dice_scores[wsi_id] = 0.0
        continue

    coords = np.array(coords)

    # Fit DBSCAN
    db = DBSCAN(eps=1.9, min_samples=7).fit(coords)
    labels = db.labels_

    # Count size of each cluster (excluding noise)
    label_counts = Counter(labels)
    valid_labels = {label for label, count in label_counts.items() if label != -1 and count >= 30}

    # Create clustered prediction mask
    pred_mask = np.zeros_like(binary_mask, dtype=np.uint8)
    for (coord, label) in zip(coords, labels):
        if label in valid_labels:
            row, col = coord
            pred_mask[row, col] = 1

    # Compute Dice coefficient
    intersection = np.logical_and(binary_mask, pred_mask).sum()
    gt_sum = binary_mask.sum()
    pred_sum = pred_mask.sum()

    if gt_sum + pred_sum == 0:
        dice = 1.0  # both empty, considered perfect match
    else:
        dice = (2.0 * intersection) / (gt_sum + pred_sum)

    dice_scores[wsi_id] = dice

    # Optional: Visualize clusters
    label_colors = {
        label: np.random.randint(0, 255, size=3)
        for label in valid_labels
    }
    cluster_img = np.zeros((h, w, 3), dtype=np.uint8)
    for (coord, label) in zip(coords, labels):
        if label in valid_labels:
            cluster_img[coord[0], coord[1]] = label_colors[label]

    plt.figure(figsize=(8, 8))
    plt.imshow(cluster_img)
    plt.title(f"DBSCAN Tumor Clusters (Filtered ≥30 patches) - {wsi_id}")
    plt.axis('off')
    plt.show() 
    
    wsi_metrics_dict[list(wsi_metrics_dict.keys())[i]] += (f", Dice coefficient after clustering = {dice:.4f}") 
    
    print("  Valid Clusters:", len(valid_labels))
    print("  Noise Points:", list(labels).count(-1)) 

In [ ]:
wsi_metrics_dict 

In [ ]:
import re
import pandas as pd

data = wsi_metrics_dict 

rows = []

pattern = re.compile(
    r'Accuracy = ([\d.]+)%, F1 Score Class 0 = ([\d.]+), F1 Score Class 1 = ([\d.]+), '
    r'Weighted F1 = ([\d.]+), Dice coefficient = ([\d.]+), Dice coefficient after clustering = ([\d.]+)'
)

for wsi_id, text in data.items():
    match = pattern.search(text)
    if match:
        acc, f1_0, f1_1, f1_weighted, dice, dice_clustered = map(float, match.groups())
        rows.append({
            'WSI': wsi_id,
            'Accuracy (%)': acc,
            'F1 Class 0': f1_0,
            'F1 Class 1': f1_1,
            'Weighted F1': f1_weighted,
            'Dice': dice,
            'Dice (Clustered)': dice_clustered
        })

df = pd.DataFrame(rows) 

In [ ]:
df.to_csv('effnet-cluster-metrics.csv', index=False) 